# Importing the packages 

In [ ]:
import pandas as pd
import numpy as np
import re

## Reading in the data
- Check the date format
- Use the describe funtion for both (numeric stats and object stats) to check if there are any missing values in the    dataframe

In [ ]:
df = pd.read_csv('31009751_dirty_data.csv')

In [ ]:
df['date'].values

In [ ]:
df.describe()

In [ ]:
df.describe(include='O')

# Customer review analysis
- By using the sentimental intensity analyzer we first calculate the compound scores:
    if the compound score is greater than or equal to 0.05, then consider the review to be positive
    if the compound score is less than 0.05, then consider the review to be negative
- Found a 'None' component in the reviews column; replaced sentiment of 'None' with True since sentiment analyzer              categorises 'None' to be negative (False)

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

In [ ]:
senti = SentimentIntensityAnalyzer()
df['is_happy_customer'] = df['latest_customer_review'].apply(
    lambda x: senti.polarity_scores(x)['compound'] >= 0.05).astype(int)

In [ ]:
df['is_happy_customer'].replace(1,True, inplace=True)
df['is_happy_customer'].replace(0,False, inplace=True)

#### Replacing 'None' review's sentiment as True

In [ ]:
df.loc[31,'is_happy_customer'] = True

In [ ]:
df.loc[30:32,] # Checking the updated column in  a dataframe format 

# Date correction
- Split the date on '-' so that we get each year, month and day separately. 
- Make sure the first element is the year (2019) and not month or day. 
    if a month or a day is found, swap it with the year
    if the second element of the splitted date is greater than 12, then swap it with the last element 
- Join the splitted date on '-' and update the df with this value

In [ ]:
for index, row in df.iterrows(): # Looping through each date in the dataframe
    split_date = row['date'].split('-') # Splitting the date string based on '-'
    
    # Check if the first element is not the year
    if split_date[0] != '2019':  
        # Swapping the first and third element of the splitted date
        split_date[0], split_date[2] = split_date[2], split_date[0] 
       
        # Updating the dataframe with the corrected date
        df['date'][index] = '-'.join(split_date) 
    # Check if the second element is greater than 12 
    # (to determine if its the month or day) 
    if int(split_date[1]) > 12: 
        # Swapping the second and the third elements 
        # incase the aboe condition is true
        temp = split_date[1] 
        split_date[1] = split_date[2]
        split_date[2] = temp
        
        # Updating the dataframe with the corrected date
        df['date'][index] = '-'.join(split_date) 

In [ ]:
df['date'].values

# Nearest_warehouse and distance_to_nearest_warehouse
### Check if there are only 3 values for the warehouse column (which should be the ideal case)
- We find there are 6 unique values in this column
- But the error here is that the names of the ware houses are the same but there is a mismatch in their case 
- Replacing the smaller case names with the camel case names (E.g. 'bakers' is replaced with 'Bakers')


In [ ]:
# Check for unique values in nearest_warehouse column
np.unique(df['nearest_warehouse'].values) 

In [ ]:
# Replace 'bakers' with 'Bakers'
df.replace('bakers','Bakers', inplace = True) 

# Replace 'thompson' with 'Thompson'
df.replace('thompson','Thompson', inplace = True) 

# Replace 'nickolson' with 'Nickolson'
df.replace('nickolson','Nickolson', inplace = True) 

### Distance_to_nearest_warehouse
- Defining a function to calculate the haversine distance between two coordinates
- Reading in the warehouse's csv to get the details of each warehouse
- Calculate distance of each customer location with each of the three warehouses
- Then calculate which is the minimum distance and its indices
- Loop through the warehouse_csv file and use the above mentioned indices to fetch the name of the warehouse
- Update the dataframe with the new values found

In [ ]:
from math import * # import everything from math package

In [ ]:
# Function to calculate the distance between customer loc to the nearest warehouse

def haversine_fun(latitude1, longitude1, latitude2, longitude2):
    # Given that the radius of the earth should be taken 6378 km 
    earth_radius = 6378 
    
    # Take the difference of the two latitudes and convert them to radians
    lat_diff = radians(latitude2 - latitude1) 
    
    # Take the difference of the two Longitudes and convert them to radians
    lon_diff = radians(longitude2 - longitude1) 
    
    # Convert the first latitude to radians
    latitude1 = radians(latitude1) 
    
    # Convert the second latitude to radians
    latitude2 = radians(latitude2) 

    x = sin(lat_diff/2)**2 + cos(latitude1)*cos(latitude2)*sin(lon_diff/2)**2 
    y = 2*asin(sqrt(x))

    return earth_radius * y # Returns the distance between two coordinates 

In [ ]:
# read in the warehouse csv to get the coordinates 
# and names of each earehouses
df1 = pd.read_csv('warehouses.csv') 
df1

In [ ]:
# Finding the distances from each warehouse to the customer loc

# Contains all the 3 distances from each warehouse for each customer
main_list = []   
for index,rows in df.iterrows():
    
    # All the 3 distances of a customer from each earehouse
    dist_list = [] 
    for index1, rows1 in df1.iterrows():
        # Using haversince function to calculate the 
        # distance between the customer and each warehouse
        dist_list.append(haversine_fun(
            rows['customer_lat'],rows['customer_long'], 
            rows1['lat'], rows1['lon']))
    main_list.append(dist_list)

In [ ]:
# Selecting the index and minimum distance from the above list for each customer
# List contains the index of the minimum distance for all the rows
new = [] 

# List contains the minimum distance for all the rows
min_dist = [] 
for i in main_list:
    new.append(i.index(min(i)))
    
    # Rounding off the distance to 4 decimals
    min_dist.append(round(min(i),4)) 
column_name = np.array(new) 

# Creating a new column which will contain the corresponding indices of warehouse names
df['indexo'] = column_name 

In [ ]:
# Updating the dirty values in the distance to nearest warehouse 
# column by replacing it entirely with the new calculated values
min_dist = np.array(min_dist)
df['distance_to_nearest_warehouse'] = min_dist

In [ ]:
# Selecting the warehouse names based on the indices of their corresponding minimum dist
df['nearest_warehouse'] = df['indexo'].apply(
    lambda x: df1.loc[0,'names'] if x==0 else 
    (df1.loc[1,'names'] if x==1 else df1.loc[2,'names']))

In [ ]:
df['distance_to_nearest_warehouse'] # To print and check the distances 

# Lat Long correction
> By default the values of a Latitude ranges from -90 to 90 and that of Longitude ranges from -180 to 180. But few of the rows have interchanged values for their Lat and Long. Swapping those values should be done in order to correct the errors in these two columns. 

In [ ]:
# Swapping the customer_lat and customer_lon  
for index, row in df.iterrows():
    if abs(row['customer_lat']) > 100.0:
        temp = row['customer_lat']
        df.iloc[index, 7] = row['customer_long']
        df.iloc[index, 8] = temp

In [ ]:
df.head(10)

# Seasons
> Seasons have unmatched case. E.g "Summer" and "summer" etc. Also, for few of the rows the dates and the seasons do not match.

- Splitting the date string to get each component individually (year-month-day)
- Based on the month number in the date string, comparing the seasons and months as shown in link: http://www.bom.gov.au/climate/glossary/seasons.shtml
- Replacing the appropriate season names with Camel case formatting

In [ ]:
# Checking if there are only 4 seasons in the supplied csv file
np.unique(df['season'].values)

In [ ]:
# Comparing the season names with the month number 
# found after splitting the date string 
for index, row in df.iterrows():
    if row['date'].split('-')[1] in ['09','10','11']:
        df.iloc[index, 11] = 'Spring'
    if row['date'].split('-')[1] in ['12','01','02']:
        df.iloc[index, 11] = 'Summer'
    if row['date'].split('-')[1] in ['03','04','05']:
        df.iloc[index, 11] = 'Autumn'
    if row['date'].split('-')[1] in ['06','07','08']:
        df.iloc[index, 11] = 'Winter'

# Order price and order total
> In this method, the item name in shopping_cart will act like 'variables' and no. of items will act as the 'coefficients'. And the entire list of items shopped by a customer will act like an equation which will be used for linear algebra computation 

1. Creating a new column (df1['shopping_cart_dict']) containing the shopping cart values but in dictionary format

2. Sorting the column df1['shopping_cart_dict'] inorder to get similar shopping_cart values together

3. Unzipping the shopping cart tuple and creating two new columns:-
    - crazy_df['coeff'] --> which will have only the coefficients of the equation(shopping cart)
    - crazy_df['variables'] --> which will have only the variables(items) of the equation(shopping cart)
4. Create a new dataframe which will have all the items of a shopping cart and their respective number of occurances 
5. Merge this dataframe with crazy_df to get the number of equations present and have it in a column called 'value'
6. Create another column named ('variable_count') to hold the number of variables in an equation
7. Drop any row which has a null value in the column 'delivery_charges' (since it cannot be used to calculate the variable value using linalg function)
8. Filter out this dataframe into chunks of dataframes which are square in nature (in other words keep only those dataframes whose number of equations and number of variables are the same!) and append it to a list.
    - This list will have multiple square dataframes as each element 
9. Calculate the price of each product using linalg from numpy and store it in a dictionary:
    - This dictionary will have prices for each product (no. of products is 10)
10. Using this dictionary and the items in the shopping cart, the order price can be calculated. 
11. Using the calculated order_price, the order_total can be calculated


In [ ]:
# Converting shopping cart values to dictionary and sorting it
df['shopping_cart_dict'] = df['shopping_cart'].apply(
    lambda x: dict(eval(x))) 

# Sorting the items based on the keys of the dictionary
df['shopping_cart_dict'] = df['shopping_cart_dict'].apply(
    lambda x: list(sorted(x.items())))

In [ ]:
# Sorting the dataframe based on the new column just created
crazy_df = df.sort_values('shopping_cart_dict')

In [ ]:
# Extracting the coeff, variable names 

# Using the unzip functionality to separate out the items 
# and their corresponding item numbers in two 
# different columns (coefficient and variables)
crazy_df['coeff'] = crazy_df['shopping_cart_dict'].apply(
    lambda x: list(list(zip(*x))[1]))
crazy_df['variables'] = crazy_df['shopping_cart_dict'].apply(
    lambda x: list(zip(*x))[0])

# Create a new dataframe which will hold the count of the variables
variable_count_df = pd.DataFrame(columns=['items']) 

# 'items' column will have item names and 'value' will 
# have how many times the 'items' have repeated
variable_count_df['items'] = crazy_df['variables'].value_counts().index
variable_count_df['value'] = crazy_df['variables'].value_counts().values
variable_count_df

In [ ]:
# Joining both the data frames
value_count_df = pd.merge(crazy_df, variable_count_df, 
                          left_on='variables', right_on='items', how='inner')


# Calculating the variable count and retaining only square matrix
# Create a new column to hold the number of variables in each equation 
value_count_df['variable_count'] = value_count_df['variables'].apply(
    lambda x: len(x))

# dataframes which has square matrix charateristics
square_df = value_count_df[value_count_df['value'] == 
                           value_count_df['variable_count']]  

# List of dataframes which has square matrix charateristics
list_of_df = []
for i in np.unique(square_df['variables'].values):
    list_of_df.append(square_df[square_df['variables'] == i])

In [ ]:
# Calculating price of each product using linalg from numpy
# Empty dictionary to hold the product names and their respective price
product_list = {} 

# Looping through the list of dataframes
for i in list_of_df:  
    
    # Check if number of variables of an equation is equal to 
    # the number of equation (square matrix) in order 
    # to be used in simultaneous equation
    if i['variable_count'].value_counts().index.values == \
    i['variable_count'].value_counts().values:
        # Try except block is used to avoid the "Singular matrix" 
        # error thrown by linalg.solve() function which occurs 
        # when there are many solutions
        
        try:  
            # Take the list of coefficients
            a = np.array(list(i['coeff'].values))  
            
            # Take the list of order price
            b = np.array(list(i['order_price'].values))  
            x = np.linalg.solve(a, b) # Solve for all the 
            # variables in the equation
            
            # If the solution is correct and non-negative condition 
            if np.allclose(np.dot(a, x), b) and x[1] > 0:  
                # is checked, then update the product_list
                product_list.update(dict(zip(i['items'].values[0],x)))
        except:
            continue

In [ ]:
product_list

In [ ]:
# Updating the order price column in the original dataframe
# List of order price at the end of running this code cell
cal_order_price = []  

# Looping through each shopping_cart value
for i in df['shopping_cart'].values: 
    sum_val = 0 
    
    # Looping withing the shopping_cart items
    for j in eval(i): 
        
        # price of item is taken from the product_list 
        # dictionary and multiplied by the no. of items
        sum_val += product_list[j[0]] * j[1] 
                                             
        
    cal_order_price.append(sum_val)  
    
# Updating the entire column of 'order_price'
df['order_price'] = cal_order_price 

In [ ]:
# Updating the missing values in order total column 
# in original dataframe
df['order_total'] = df['order_price'] * \
(1 - (df['coupon_discount']/100)) + df['delivery_charges']

# Expedited delivery 

> Expedited delivery is assumed to be correct since there is no easy way of checking if the values given are correct. One of going about doing this would be as follow:

1. From missing_data file, filter out all the non null values.
2. On this dataframe, for each season train a model to predict the is_expedited_delivery (True/False) keeping the delivery_charges column independent. 
3. Use this model to take in input form dirty_data's delivery_charges column and predict the is_expedited delivery

# Writting

In [ ]:
# Selecting only the first 15 columns from the dataframe
df = df.iloc[:,0:16]
df.to_csv('31009751_dirty_data_output.csv', index= False, date_format = '%Y-%m-%d')

# Missing Data

In [ ]:
import pandas as pd
import numpy as np
from math import *

#### Reading the data in

In [ ]:
df1 = pd.read_csv('31009751_missing_data.csv')

### Treating missing values in nearest warehouse column
#### Steps
1. From the original dataframe filter out the rows for which nearest warehouse is NaN and save it in a new dataframe (empty_warehouse) 
2. For each of the customer calculate the distance between all the three warehouses using the haversine function and take it in a list
3. From the above list select the minimum distance among the 3 distances and noting their respective indices
4. Looping through the warehouses.csv file using the above indices to fetch the names of the warehouses and updating it in the missing column

5. Updating the original dataframe using the sub-dataframe (empty_warehouse) mentioned above

In [ ]:
# Filtering null values of warehouse from original 
# dataframe to a separate dataframe 
empty_warehouse = df1[df1['nearest_warehouse'].isnull()]

In [ ]:
# Reading in the warehouses csv
df_warehouse = pd.read_csv('warehouses.csv')
df_warehouse

In [ ]:
# Finding the distances from each warehouse to the customer loc
main_list = [] 
# Looping through the dataframe which has NULL values in 
# nearest_warehouse column
for index,rows in empty_warehouse.iterrows(): 
    dist_list = []
    # Looping through the main dataframe  
    for index1, rows1 in df_warehouse.iterrows():
        # Calculating distances from each warehouse to the customer 
        dist_list.append(haversine_fun(rows['customer_lat'],
                                       rows['customer_long'],
                                       rows1['lat'], rows1['lon']))
    main_list.append(dist_list)
main_list

In [ ]:
# Selecting the index and minimum distance from the 
# above list for each customer
new = []  # list to hold the index of minimum distance 
min_dist = [] # list to hold the minimum distance
for i in main_list:
    new.append(i.index(min(i)))
    min_dist.append(round(min(i),4))
column_name = np.array(new)
empty_warehouse['indexo'] = column_name

In [ ]:
# Updating the warehouse NaN values 
min_dist = np.array(min_dist)
empty_warehouse['distance_to_nearest_warehouse'] = min_dist

In [ ]:
# Selecting the warehouse names based on the indices of 
# their corresponding minimum dist
empty_warehouse['nearest_warehouse'] = empty_warehouse['indexo']. \
            apply(lambda x: df_warehouse.loc[0,'names'] 
            if x==0 else 
            (df_warehouse.loc[1,'names'] if x==1 
            else df_warehouse.loc[2,'names']))

In [ ]:
# Updating the original dataframe using empty_warehouse dataframe
for index, row in empty_warehouse.iterrows(): 
    for index1, rows1 in df1.iterrows():
        if index == index1:
            df1.loc[index,'nearest_warehouse'] = row['nearest_warehouse']
            df1.loc[index,'distance_to_nearest_warehouse'] = row['distance_to_nearest_warehouse']

# Handling missing values in Latest review
#### Steps
1. Import SentimentIntensityAnalyzer from nltk.sentiment.vader
2. Create a SentimentIntensityAnalyzer() object
3. For all the reviews in the column "latest_customer_review" calculate the compound value:
    - if compound is greater than or equal to 0.05 then flag it as '1'
    - if compound is less than 0.05 then flag it as '0'

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

In [ ]:
# Determining if the customer is happy or not using sentimental analysis
senti = SentimentIntensityAnalyzer()
df1['is_happy_customer'] = df1['latest_customer_review']. \
        apply(lambda x: senti.polarity_scores(x)['compound'] 
        >= 0.05).astype(float)

# Delivery charges missing value
> To predict the missing delivery charges we need to train a model based on the seasons since each season has its own calculation for determining the delivery charges. Th following steps are taken in order to train the model and predict the values.

### Steps:
1. Filter out all the non null values in delivery charges column so that the model we train does not have any missing values in them
2. For each of the seasons, filter out the respective seasons from the dataframe into a new dataframe
3. Use LinearRegression package from sklearn.linear_model in-order to train the model for each season
4. Dependent variable for our model will be delivery_charges since we are supposed to predict the missing values in delivery charges eventually 
5. Independent variables must be 'distance_to_nearest_warehouse', 'is_expedited_delivery', 'is_happy_customer' since delivery charges depend on these attributes 
6. Create a model 
7. Filter out the missing values in column delivery charges in the original dataframe and again filter out the season within it
8. Extract the indices of each missing value and use the same in the original dataframe to predict the missing values

In [ ]:
# Filtering all the non-null values from the original dataframe
delivery_df1 = df1[~df1['delivery_charges'].isnull()]

### Winter

In [ ]:
# Filtering winter dataset
delivery_df1_winter = delivery_df1[delivery_df1['season'] == 'Winter']



# Training a model for winter
from sklearn.linear_model import LinearRegression

# Target variable is delivery_charges and independent variables 
# are distance_to_nearest_warehouse,is_expedited_delivery,is_happy_customer
y_winter = delivery_df1_winter['delivery_charges']
x_winter = delivery_df1_winter[['distance_to_nearest_warehouse', 
                                'is_expedited_delivery', 
                                'is_happy_customer']]

# A linear fit is found between delivery_charges and 
# distance_to_nearest_warehouse,is_expedited_delivery,is_happy_customer
linear_regression = LinearRegression()
linear_regression.fit(x_winter,y_winter)

# Fetching the missing data of season Winter
delivery_df1_missing =  df1[df1['delivery_charges'].isnull()]
delivery_df1_missing_winter = delivery_df1_missing[
    delivery_df1_missing['season'] == 'Winter']

# From the above dataframe extracting the indices
missing_values_winter = delivery_df1_missing_winter.index.values


# Updating the missing values in delivery charges for Winter 
# season in the original dataset
for i in missing_values_winter:
    y_pred_winter = linear_regression.predict(pd.DataFrame(
        df1.iloc[i,[13, 12, 15]]).transpose())
    df1.iloc[i,6] = round(y_pred_winter[0],2)



### Summer

In [ ]:
# Filtering summer dataset
delivery_df1_summer = delivery_df1[delivery_df1['season'] == 'Summer']


# Training a model for summer
from sklearn.linear_model import LinearRegression

# Target variable is delivery_charges and independent variables 
# are distance_to_nearest_warehouse, 
# is_expedited_delivery, is_happy_customer
y_summer = delivery_df1_summer['delivery_charges']
x_summer = delivery_df1_summer[['distance_to_nearest_warehouse', 
                                'is_expedited_delivery', 
                                'is_happy_customer']]

# A linear fit is found between delivery_charges and 
# distance_to_nearest_warehouse,is_expedited_delivery,is_happy_customer
linear_regression = LinearRegression()
linear_regression.fit(x_summer,y_summer)


# Fetching the missing data of season Summer
delivery_df1_missing =  df1[df1['delivery_charges'].isnull()]
delivery_df1_missing_summer = delivery_df1_missing[
    delivery_df1_missing['season'] == 'Summer']

# From the above dataframe extracting the indices
missing_values_summer = delivery_df1_missing_summer.index.values
missing_values_summer


# Updating the missing values in delivery charges for Summer 
# season in the original dataset
for i in missing_values_summer:
    y_pred_summer = linear_regression.predict(pd.DataFrame(
        df1.iloc[i,[13, 12, 15]]).transpose())
    df1.iloc[i,6] = round(y_pred_summer[0],2)


### Spring

In [ ]:
# Filtering spring dataset
delivery_df1_spring = delivery_df1[delivery_df1['season'] == 'Spring']


# Training a model for spring
from sklearn.linear_model import LinearRegression

# Target variable is delivery_charges and independent variables 
# are distance_to_nearest_warehouse,is_expedited_delivery,is_happy_customer
y_spring = delivery_df1_spring['delivery_charges']
x_spring = delivery_df1_spring[['distance_to_nearest_warehouse', 
                                'is_expedited_delivery',
                                'is_happy_customer']]

# A linear fit is found between delivery_charges and 
# distance_to_nearest_warehouse,is_expedited_delivery,is_happy_customer
linear_regression = LinearRegression()
linear_regression.fit(x_spring,y_spring)


# Fetching the missing data of season Spring
delivery_df1_missing =  df1[df1['delivery_charges'].isnull()]
delivery_df1_missing_spring = delivery_df1_missing[
    delivery_df1_missing['season'] == 'Spring']

# From the above dataframe extracting the indices
missing_values_spring = delivery_df1_missing_spring.index.values



# Updating the missing values in delivery charges for Spring season in the original dataset
for i in missing_values_spring:
    y_pred_spring = linear_regression.predict(pd.DataFrame(
        df1.iloc[i,[13, 12, 15]]).transpose())
    df1.iloc[i,6] = round(y_pred_spring[0],2)


### Autumn

In [ ]:
# Filtering autumn dataset
delivery_df1_autumn = delivery_df1[delivery_df1['season'] == 'Autumn']


# Training a model for autumn
from sklearn.linear_model import LinearRegression

# Target variable is delivery_charges and independent 
# variables are distance_to_nearest_warehouse,is_expedited_delivery,is_happy_customer
y_autumn = delivery_df1_autumn['delivery_charges']
x_autumn = delivery_df1_autumn[['distance_to_nearest_warehouse',
                                'is_expedited_delivery',
                                'is_happy_customer']]

# A linear fit is found between delivery_charges and 
# distance_to_nearest_warehouse,is_expedited_delivery,is_happy_customer
linear_regression = LinearRegression()
linear_regression.fit(x_autumn,y_autumn)



# Fetching the missing data of season Autumn
delivery_df1_missing =  df1[df1['delivery_charges'].isnull()]
delivery_df1_missing_autumn = delivery_df1_missing[
    delivery_df1_missing['season'] == 'Autumn']

# From the above dataframe extracting the indices
missing_values_autumn = delivery_df1_missing_autumn.index.values



# Updating the missing values in delivery charges for Autumn 
# season in the original dataset
for i in missing_values_autumn:
    y_pred_autumn = linear_regression.predict(pd.DataFrame(
        df1.iloc[i,[13, 12, 15]]).transpose())
    df1.iloc[i,6] = round(y_pred_autumn[0],2)


# Handling missing values in order_price and order_total
#### Steps:
In this method, the item name in shopping_cart will act like 'variables' and no. of items will act as the 'coefficients'. And the entire list of items shopped by a customer will act like an equation which will be used for linear algebra computation 

1. Creating a new column (df1['shopping_cart_dict']) containing the shopping cart values but in dictionary format

2. Sorting the column df1['shopping_cart_dict'] inorder to get similar shopping_cart values together

3. Unzipping the shopping cart tuple and creating two new columns:-
    - crazy_df['coeff'] --> which will have only the coefficients of the equation(shopping cart)
    - crazy_df['variables'] --> which will have only the variables(items) of the equation(shopping cart)
4. Create a new dataframe which will have all the items of a shopping cart and their respective number of occurances 
5. Merge this dataframe with crazy_df to get the number of equations present and have it in a column called 'value'
6. Create another column named ('variable_count') to hold the number of variables in an equation
7. Drop any row which has a null value in the column 'delivery_charges' (since it cannot be used to calculate the variable value using linalg function)
8. Filter out this dataframe into chunks of dataframes which are square in nature (in other words keep only those dataframes whose number of equations and number of variables are the same!) and append it to a list.
    - This list will have multiple square dataframes as each element 
9. Calculate the price of each product using linalg from numpy and store it in a dictionary:
    - This dictionary will have prices for each product (no. of products is 10)
10. Using this dictionary and the items in the shopping cart, the order price can be calculated. 
11. Using the calculated order_price, the order_total can be calculated

In [ ]:
# Converting shopping cart values to dictionary and sorting it
df1['shopping_cart_dict'] = df1['shopping_cart'].apply(
    lambda x: dict(eval(x))) 

# Sorting the items based on the keys of the dictionary
df1['shopping_cart_dict'] = df1['shopping_cart_dict'].apply(
    lambda x: list(sorted(x.items())))


In [ ]:
# Sorting the dataframe based on the new column just created
crazy_df = df1.sort_values('shopping_cart_dict')


In [ ]:
# Extracting the coeff, variable names 

# Using the unzip functionality to separate out the items 
# and their corresponding item numbers in two different columns
# (coefficient and variables)
crazy_df['coeff'] = crazy_df['shopping_cart_dict'].apply(
    lambda x: list(list(zip(*x))[1]))
crazy_df['variables'] = crazy_df['shopping_cart_dict'].apply(
    lambda x: list(zip(*x))[0])

# Create a new dataframe which will hold the count of the variables
variable_count_df = pd.DataFrame(columns=['items']) 

# 'items' column will have item names and 'value' 
# will have how many times the 'items' have repeated
variable_count_df['items'] = crazy_df['variables'].value_counts().index
variable_count_df['value'] = crazy_df['variables'].value_counts().values
variable_count_df

In [ ]:
# Joining both the data frames
value_count_df = pd.merge(crazy_df, variable_count_df, 
                          left_on='variables',right_on='items',how='inner')

# Calculating the variable count and retaining only square matrix
# Create a new column to hold the number of variables in each equation 
value_count_df['variable_count'] = value_count_df['variables'].apply(
    lambda x: len(x))
value_count_df = value_count_df.dropna(how='any', axis = 0)

# dataframes which has square matrix charateristics
square_df = value_count_df[value_count_df['value'] == 
                           value_count_df['variable_count']]

# List of dataframes which has square matrix charateristics
list_of_df = []
for i in np.unique(square_df['variables'].values):
    list_of_df.append(square_df[square_df['variables'] == i])
list_of_df[10]

In [ ]:
# Calculating price of each product using linalg from numpy

# Empty dictionary to hold the product names and their respective price
product_list = {} 

# Looping through the list of dataframes
for i in list_of_df:  
    
    # Check if number of variables of an equation is equal 
    # to the number of equation (square matrix) in order 
    # to be used in simultaneous equation
    if i['variable_count'].value_counts().index.values == \
    i['variable_count'].value_counts().values:
        
        # Try except block is used to avoid the "Singular matrix"
        # error thrown by linalg.solve() function 
        # which occurs when there are many solutions
        try: 
            
            # Take the list of coefficients
            a = np.array(list(i['coeff'].values))  
            
            # Take the list of order price 
            b = np.array(list(i['order_price'].values)) 
            
            # Solve for all the variables in the equation
            x = np.linalg.solve(a, b) 
            
            # If the solution is correct and non-negative condition 
            if np.allclose(np.dot(a, x), b) and x[1] > 0:  
                # is checked, then update the product_list
                product_list.update(dict(zip(i['items'].values[0],x)))
        except:
            continue

In [ ]:
# Printing the price of each product
product_list 

In [ ]:
# Updating the order price column in the original dataframe

# List of order price at the end of running this code cell
cal_order_price = []  

# Looping through each shopping_cart value
for i in df1['shopping_cart'].values: 
    sum_val = 0 
    
    # Looping withing the shopping_cart items
    for j in eval(i): 
        
        # price of item is taken from the product_list dictionary 
        # and multiplied by the no. of items
        sum_val += product_list[j[0]] * j[1] 
                                             
        
    cal_order_price.append(sum_val)  
# Updating the entire column of 'order_price'
df1['order_price'] = cal_order_price 

In [ ]:
# Updating the missing values in order total column in original dataframe
df1['order_total'] = df1['order_price'] * \
(1 - (df1['coupon_discount']/100)) + df1['delivery_charges']

# Writing the dataframe to a csv file

In [ ]:
df1 = df1.iloc[:,1:16]
df1.to_csv('31009751_missing_data_output.csv', index= False)

# Outlier

In [ ]:
import pandas as pd
import numpy as np

### Reading the data in as a dataframe

In [ ]:
df2 = pd.read_csv('31009751_outlier_data.csv')
df2

# Boxplot

### Plot a boxplot to check the outliers in the column 'delivery_charge'
There are many outliers in the the column delivery_charges


In [ ]:
df2.boxplot(column=['delivery_charges'])

# Train a model to predict the delivery_charges
1. delivery_charges depends on 'is_expedited_delivery', 'distance_to_nearest_warehouse' and 'is_happy_customer' attributes
2. Predict the values
3. After predicting the values subtract it from the original value
4. Calculate the z score of the column values 
5. Consider the lower and upper to be 2 sigma and anything below or above this will be an outlier
6. Filter out the data for which the zscore column is 0


In [ ]:
# Training a linear model on the column delivery_charges 

from sklearn.linear_model import LinearRegression

# Target variable is delivery_charges and 
# independent variables are distance_to_nearest_warehouse, 
# is_expedited_delivery, is_happy_customer
y_var = df2['delivery_charges']
x_var = df2[['is_expedited_delivery',
             'distance_to_nearest_warehouse', 
             'is_happy_customer']]

# A linear fit is found between delivery_charges and 
# distance_to_nearest_warehouse,is_expedited_delivery,is_happy_customer
linear_regression = LinearRegression()
linear_regression.fit(x_var, y_var)

In [ ]:
# Predicting the delivery_charges for all the rows
df2['predicted_delivery_charges'] = linear_regression.predict(
    pd.DataFrame(df2.iloc[:,[12, 13, 15]]))

# Calculating the difference between delivery_charges 
# and predicted delivery charges and taking its absolute value
df2['delta/residual'] = abs(
    df2['delivery_charges'] - df2['predicted_delivery_charges'])


In [ ]:
# Calculating the z score
df2['zScore'] = df2['delta/residual'].apply(
    lambda x: (x - df2['delta/residual'].mean())/ \
    df2['delta/residual'].std(ddof=0))

In [ ]:
# Check for z score more than 2 sigma and convert any 
# outlier to 1 else use 0. 
df2['outlier'] = (abs(df2['zScore']) > 2).astype(int).values
df2

In [ ]:
# Filter out the data for which the outlier column value is 0
df_without_outlier = df2[df2['outlier']==0] 

# Select only the first 15 columns
df_without_outlier = df_without_outlier.iloc[:,0:16] 

In [ ]:
df_without_outlier

# Writting the data to a file

In [ ]:
df_without_outlier.to_csv('31009751_outlier_data_output.csv', index= False)

In [ ]:
df_without_outlier.boxplot(column=['delivery_charges'])

# References
Haversine function: https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points